In [1]:
# Step 1: Traffic Light Detection Only
!pip install ultralytics opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:00


In [18]:
from ultralytics import YOLO
import cv2, numpy as np, torch, warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load model
model = YOLO("yolov8m.pt")

# --- find class id for 'traffic light' ---
def find_class_id(names, target="traffic light"):
    if isinstance(names, dict):
        for k, v in names.items():
            if str(v).lower() == target.lower():
                return int(k)
    else:
        try:
            return int(list(names).index(target))
        except ValueError:
            return None
    return None

traffic_id = find_class_id(model.names, "traffic light")
if traffic_id is None:
    for alt in ["traffic_light", "trafficlight"]:
        traffic_id = find_class_id(model.names, alt)
        if traffic_id is not None:
            break
if traffic_id is None:
    raise ValueError(f"'traffic light' not found in model.names")

print("Traffic-light class id:", traffic_id)
device = '0' if torch.cuda.is_available() else 'cpu'


# --------------------------
# IMAGE WORKFLOW
# --------------------------
def process_image(img_path):
    results = model.predict(
        source=img_path,
        conf=0.25,
        imgsz=640,
        classes=[traffic_id],
        device=device,
        verbose=False
    )
    r = results[0]
    img = cv2.imread(img_path)

    if hasattr(r, "boxes") and len(r.boxes) > 0:
        xyxy = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        for i in range(len(xyxy)):
            x1,y1,x2,y2 = xyxy[i].astype(int)
            conf = confs[i]

            crop = img[y1:y2, x1:x2]
            color = get_signal_color(crop)

            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
            label = f"{color} {conf:.2f}"
            cv2.putText(img, label, (x1, max(0,y1-10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    from google.colab.patches import cv2_imshow
    cv2_imshow(img)



# --------------------------
# VIDEO WORKFLOW
# --------------------------
def process_video(video_path, save_path="output.avi"):
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or np.isnan(fps):
        fps = 25

    w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(save_path, fourcc, fps, (w,h))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        results = model.predict(
            source=frame,
            conf=0.25,
            imgsz=640,
            classes=[traffic_id],
            device=device,
            verbose=False
        )
        r = results[0]

        if hasattr(r, "boxes") and len(r.boxes) > 0:
            xyxy = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            for i in range(len(xyxy)):
                x1,y1,x2,y2 = xyxy[i].astype(int)
                conf = confs[i]

                crop = frame[y1:y2, x1:x2]
                color = get_signal_color(crop)

                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(frame, f"{color} {conf:.2f}",
                            (x1, max(0,y1-10)), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (255,255,255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ Processed {frame_count} frames. Saved to {save_path}")



# Example usage:
# process_image("/content/traffic light.png")

process_video("/content/VDO Traffic Lights 2 seconds green.mp4",
              save_path="/content/detected_output.avi")

Traffic-light class id: 9
✅ Processed 1171 frames. Saved to /content/detected_output.avi


In [19]:
from google.colab import files
files.download("/content/detected_output.avi")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
def get_signal_color(crop_bgr):
    """
    Classify the traffic light signal color (red, yellow, green) using HSV masks.
    """
    hsv = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2HSV)

    # HSV ranges for colors
    red1 = cv2.inRange(hsv, (0, 70, 50), (10, 255, 255))
    red2 = cv2.inRange(hsv, (170, 70, 50), (180, 255, 255))
    red_mask = cv2.bitwise_or(red1, red2)

    yellow_mask = cv2.inRange(hsv, (15, 70, 50), (35, 255, 255))
    green_mask  = cv2.inRange(hsv, (40, 70, 50), (90, 255, 255))

    # Count non-zero pixels
    r, y, g = cv2.countNonZero(red_mask), cv2.countNonZero(yellow_mask), cv2.countNonZero(green_mask)

    if max(r, y, g) == 0:
        return "unknown"
    if r > y and r > g:
        return "red"
    elif y > r and y > g:
        return "yellow"
    else:
        return "green"
